In [1]:
import os
import datetime
import tensorflow as tf
import numpy as np

In [2]:
def ConvLSTMAutoEncoder(input_shape=(20, 64, 64, 1)):
    model = tf.keras.models.Sequential(name="ConvLSTMAutoEncoder")
    
    # Encoder: ConvLSTM
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', return_sequences=True, input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(tf.keras.layers.BatchNormalization())
        
    # Decoder: ConvLSTM
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ConvLSTM2D(filters=1, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(tf.keras.layers.BatchNormalization())
    
    return model

In [3]:
# Load MovingMNIST Data
mnist_test_seq = np.load("./data/mnist_test_seq.npy")
x = mnist_test_seq.reshape(10000, 20, 64, 64, 1)
x.shape

(10000, 20, 64, 64, 1)

In [4]:
model = ConvLSTMAutoEncoder(input_shape=(20, 64, 64, 1))
model.compile(loss='mse', optimizer='adadelta')
model.summary()

date_string = datetime.datetime.now().strftime('%Y%m%d %H:%M:%S')
os.mkdir('./log/'+date_string)
print(date_string)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "ConvLSTMAutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 20, 64, 64, 128)   594944    
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 64, 64, 128)   512       
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 20, 64, 64, 128)   1180160   
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 64, 64, 128)   512       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 20, 64, 64, 128)   1180160   
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 64, 64, 128)   512   

In [6]:
callbacks=[]
callbacks.append(tf.keras.callbacks.ModelCheckpoint(filepath='./log/'+date_string+'/bestweights.hdf5', save_best_only=True))
model.fit(x, x, epochs=50, batch_size=4, callbacks=callbacks)

Epoch 1/50
 1208/10000 [==>...........................] - ETA: 48:27 - loss: 2745.5200

KeyboardInterrupt: 